In [1]:
import duckdb

In [2]:
con = duckdb.connect(config={"allow_unsigned_extensions": "true"})
con.install_extension("./erpl.duckdb_extension");
con.load_extension("erpl");
con.sql("""
SET sap_ashost = 'localhost';
SET sap_sysnr = '00';
SET sap_user = 'DEVELOPER';
SET sap_password = 'Htods70334';
SET sap_client = '001';
SET sap_lang = 'EN';
""");

-- Loading ERPL Trampoline Extension. --
(The purpose of the extension is to extract dependencies and load the ERPL implementation)
Saving ERPL SAP dependencies to '/home/jr/.duckdb/extensions/v0.9.2/linux_amd64' and loading them ... done
ERPL extension extracted and saved to /home/jr/.duckdb/extensions/v0.9.2/linux_amd64.
ERPL implementation extension installed from /home/jr/.duckdb/extensions/v0.9.2/linux_amd64/erpl_rfc.duckdb_extension.
ERPL implementation extension loaded. For instructions how to use it visit https://erpl.io


In [3]:
con.sql("SELECT * FROM duckdb_functions() WHERE function_name LIKE '%sap_%';")

┌───────────────┬─────────────┬──────────────────────┬───────────────┬───┬──────────┬──────────────┬─────────┐
│ database_name │ schema_name │    function_name     │ function_type │ … │ internal │ function_oid │ example │
│    varchar    │   varchar   │       varchar        │    varchar    │   │ boolean  │    int64     │ varchar │
├───────────────┼─────────────┼──────────────────────┼───────────────┼───┼──────────┼──────────────┼─────────┤
│ system        │ main        │ sap_bics_show_quer…  │ table         │ … │ true     │         1434 │ NULL    │
│ system        │ main        │ sap_odp_read_full    │ table         │ … │ true     │         1464 │ NULL    │
│ system        │ main        │ sap_odp_show_cursors │ table         │ … │ true     │         1462 │ NULL    │
│ system        │ main        │ sap_odp_show_subsc…  │ table         │ … │ true     │         1460 │ NULL    │
│ system        │ main        │ sap_odp_preview      │ table         │ … │ true     │         1458 │ NULL    │
│

In [4]:
con.sql("SELECT * FROM sap_odp_show_contexts();")

┌────────────────┬──────────────────────────────────┬─────────┐
│ technical_name │               text               │ release │
│    varchar     │             varchar              │ varchar │
├────────────────┼──────────────────────────────────┼─────────┤
│ ABAP_CDS       │ ABAP Core Data Services          │ SBC754  │
│ BW             │ SAP NetWeaver Business Warehouse │ SBW754  │
│ HANA           │ HANA Information Views           │         │
│ SAPI           │ DataSources/Extractors           │ SDE754  │
└────────────────┴──────────────────────────────────┴─────────┘

In [5]:
con.sql("SELECT * FROM sap_odp_show('ABAP_CDS') WHERE technical_name LIKE '%EPM%' AND SEMANTICS NOT IN ('T');")

┌────────────────────┬─────────────────────────────────────────────┬───────────┬────────────────────────┐
│   technical_name   │                    text                     │ semantics │     semantics_text     │
│      varchar       │                   varchar                   │  varchar  │        varchar         │
├────────────────────┼─────────────────────────────────────────────┼───────────┼────────────────────────┤
│ REPMDEP_VALEND_V$F │ REPM_EMP_DEPT_VALIDEND                      │ F         │ Transaction Data/Facts │
│ REPM_DEPHV2$H      │ Department Hierarchy View                   │ H         │ Hierarchy              │
│ REPM_DEPHV3$H      │ Department Hierarchy                        │ H         │ Hierarchy              │
│ REPM_DEPTD$P       │ Department Details                          │ P         │ Master Data/Attributes │
│ REPM_DEPTHV1$E     │ Department Hierarchy View                   │ E         │ E                      │
│ REPM_DV1$P         │ Department View        

In [6]:
con.sql("""
SELECT technical_name, COLUMNS(supports_), parent_node, FROM sap_odp_describe('ABAP_CDS', 'REPM_EMP$P')
""")

┌────────────────┬───────────────┬────────────────┬────────────────────┬───────────────┐
│ technical_name │ supports_full │ supports_delta │ supports_real_time │  parent_node  │
│    varchar     │    boolean    │    boolean     │      boolean       │    varchar    │
├────────────────┼───────────────┼────────────────┼────────────────────┼───────────────┤
│ REPM_EMP       │ true          │ false          │ false              │ BW-BCT-NW-NWD │
└────────────────┴───────────────┴────────────────┴────────────────────┴───────────────┘

In [7]:
con.sql("""
WITH
    meta AS MATERIALIZED (SELECT * FROM sap_odp_describe('ABAP_CDS', 'SEPM_ISOI$P')),
    txt AS (SELECT unnest(texts, recursive:=true) FROM meta),
    fields AS (SELECT unnest(fields, recursive:=true) FROM meta)
SELECT * FROM fields LEFT JOIN txt ON (fields.technical_name = txt.technical_name) WHERE txt.language = 'E'
""")

┌──────────────────────┬──────────────────────┬───────────┬───┬──────────────────┬─────────────┬──────────────────────┐
│    technical_name    │         text         │ abap_type │ … │    text_short    │ text_medium │      text_long       │
│       varchar        │       varchar        │  varchar  │   │     varchar      │   varchar   │       varchar        │
├──────────────────────┼──────────────────────┼───────────┼───┼──────────────────┼─────────────┼──────────────────────┤
│ SALESORDERITEMUUID   │ EPM: Generic Node …  │ RAW       │ … │ Node Key         │             │ EPM: Generic Node …  │
│ SALESORDERUUID       │ EPM: Sales Order U…  │ RAW       │ … │ Sales Order UUID │             │ EPM: Sales Order U…  │
│ SALESORDERITEM       │ EPM: Sales Order I…  │ CHAR      │ … │ Item Position    │             │ EPM: Sales Order I…  │
│ PRODUCTUUID          │ EPM: Generic Node …  │ RAW       │ … │ Node Key         │             │ EPM: Generic Node …  │
│ SHORTTEXTGROUPUUID   │ EPM: Generic No

In [8]:
con.sql("SELECT * FROM (SELECT unnest(texts, recursive:=true) FROM sap_odp_describe('ABAP_CDS', 'SEPM_ISUPP$P')) AS SS WHERE language = 'D'")

┌────────────────────────┬──────────┬─────────────────────┬─────────────┬──────────────────────────────────────────┐
│     technical_name     │ language │     text_short      │ text_medium │                text_long                 │
│        varchar         │ varchar  │       varchar       │   varchar   │                 varchar                  │
├────────────────────────┼──────────┼─────────────────────┼─────────────┼──────────────────────────────────────────┤
│                        │ D        │                     │             │ EPM-Demo: Geschäftspartner               │
│ ADDRESSUUID            │ D        │ Knotenschlüssel     │             │ EPM: Generischer Knotenschlüssel         │
│ COMPANYNAME            │ D        │ Firma               │             │ EPM: Firmenname                          │
│ CREATEDBYUSER          │ D        │ Knotenschlüssel     │             │ EPM: Generischer Knotenschlüssel         │
│ CREATIONDATETIME       │ D        │ Angelegt um         │     

In [9]:
con.sql("""SELECT technical_name, text, abap_type, length, decimals from (SELECT unnest(fields, recursive:=true) FROM sap_odp_describe('ABAP_CDS', 'SEPM_IBUPA$P')) AS SS""")

┌─────────────────────┬─────────────────────────────────────────────────┬───────────┬────────┬──────────┐
│   technical_name    │                      text                       │ abap_type │ length │ decimals │
│       varchar       │                     varchar                     │  varchar  │ int64  │  int64   │
├─────────────────────┼─────────────────────────────────────────────────┼───────────┼────────┼──────────┤
│ BUSINESSPARTNERUUID │ EPM: Business Partner Key                       │ RAW       │     16 │        0 │
│ BUSINESSPARTNER     │ EPM: Business Partner ID                        │ CHAR      │     10 │        0 │
│ BUSINESSPARTNERROLE │ EPM: Business Partner Role                      │ CHAR      │      3 │        0 │
│ CURRENCY            │ EPM: Currency Code                              │ CUKY      │      5 │        0 │
│ COMPANYNAME         │ EPM: Company Name                               │ CHAR      │     80 │        0 │
│ LEGALFORM           │ EPM: Legal Form of Bus

In [7]:
con.sql("SELECT *  FROM sap_odp_preview('ABAP_CDS', 'SEPM_ISOI$P', MAX_ROWS=10)")

┌──────────────────────┬──────────────────────┬────────────────┬───┬─────────────────┬────────────────┬────────────────┐
│  SALESORDERITEMUUID  │    SALESORDERUUID    │ SALESORDERITEM │ … │ OPPORTUNITYITEM │ ODQ_CHANGEMODE │ ODQ_ENTITYCNTR │
│         blob         │         blob         │    varchar     │   │     varchar     │    varchar     │ decimal(19,0)  │
├──────────────────────┼──────────────────────┼────────────────┼───┼─────────────────┼────────────────┼────────────────┤
│ \x0Aq\xE5\xD7\xE6U…  │ \x0Aq\xE5\xD7\xE6U…  │ 0000000010     │ … │ NULL            │ NULL           │           NULL │
│ \x0Aq\xE5\xD7\xE6U…  │ \x0Aq\xE5\xD7\xE6U…  │ 0000000010     │ … │ NULL            │ NULL           │           NULL │
│ \x0Aq\xE5\xD7\xE6U…  │ \x0Aq\xE5\xD7\xE6U…  │ 0000000010     │ … │ NULL            │ NULL           │           NULL │
│ \x0Aq\xE5\xD7\xE6U…  │ \x0Aq\xE5\xD7\xE6U…  │ 0000000010     │ … │ NULL            │ NULL           │           NULL │
│ \x0Aq\xE5\xD7\xE6U…  │ \x0Aq\x

In [12]:
con.sql("SELECT * FROM sap_odp_show_subscriptions(ERPL_ONLY=false)")

┌────────────┬─────────────────┬─────────────────┬─────────────────┐
│ queue_name │ subscriber_type │ subscriber_name │ subscriber_proc │
│  varchar   │     varchar     │     varchar     │     varchar     │
├──────────────────────────────────────────────────────────────────┤
│                              0 rows                              │
└──────────────────────────────────────────────────────────────────┘

In [13]:
con.sql("SELECT * FROM sap_odp_show_cursors(ERPL_ONLY=false, SUBSCRIBER_NAME='XU')")

┌────────────┬─────────────────┬───────────────┬───────────────┬───────────┬────────────────────┬──────────────┐
│ queue_name │ subscriber_proc │ subscriber_id │    pointer    │ is_closed │ is_delta_extension │ request_date │
│  varchar   │     varchar     │    varchar    │ decimal(23,9) │  boolean  │      boolean       │  timestamp   │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                    0 rows                                                    │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────┘